---

* 출처: LangChain 공식 문서 또는 해당 교재명
* 원본 URL: https://smith.langchain.com/hub/teddynote/summary-stuff-documents

---

### **5. `05. LLM-as-Judge`**

#### **2) `Question-Answer Evaluator`**

* 가장 기본 기능을 가진 `Evaluator` = `Query` - `Answer` 평가하기

* 사용자 입력 = `input` → `LLM`이 생성한 답변 → `prediction`으로 정답 답변은 `reference`로 정의됨

  * `Prompt`변수 
    * `query`: 질문
    * `result`: `LLM` 답변
    * `answer`: 정답 답변

* **새로운 가상환경 생성 - `lc_eval_env`**

  * `Python-3.12`
  * `Pydantic`: `ver 1.10.18`

In [1]:
import pydantic
print(f"Pydantic 버전: {pydantic.__version__}")

Pydantic 버전: 1.10.18


<small>

* **`Pydantic 버전: 1.10.18`** - (`0.1s`)

---

* **`환경 설정`**

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()                           # True

True

In [2]:
from langsmith import Client
from langsmith import traceable

import os

# LangSmith 환경 변수 확인

print("\n--- LangSmith 환경 변수 확인 ---")
langchain_tracing_v2 = os.getenv('LANGCHAIN_TRACING_V2')
langchain_project = os.getenv('LANGCHAIN_PROJECT')
langchain_api_key_status = "설정됨" if os.getenv('LANGCHAIN_API_KEY') else "설정되지 않음" # API 키 값은 직접 출력하지 않음

if langchain_tracing_v2 == "true" and os.getenv('LANGCHAIN_API_KEY') and langchain_project:
    print(f"✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='{langchain_tracing_v2}')")
    print(f"✅ LangSmith 프로젝트: '{langchain_project}'")
    print(f"✅ LangSmith API Key: {langchain_api_key_status}")
    print("  -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.")
else:
    print("❌ LangSmith 추적이 완전히 활성화되지 않았습니다. 다음을 확인하세요:")
    if langchain_tracing_v2 != "true":
        print(f"  - LANGCHAIN_TRACING_V2가 'true'로 설정되어 있지 않습니다 (현재: '{langchain_tracing_v2}').")
    if not os.getenv('LANGCHAIN_API_KEY'):
        print("  - LANGCHAIN_API_KEY가 설정되어 있지 않습니다.")
    if not langchain_project:
        print("  - LANGCHAIN_PROJECT가 설정되어 있지 않습니다.")


--- LangSmith 환경 변수 확인 ---
✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='true')
✅ LangSmith 프로젝트: 'LangChain-prantice'
✅ LangSmith API Key: 설정됨
  -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.


<small>

* 셀 출력

    ```bash
    --- LangSmith 환경 변수 확인 ---
    ✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='true')
    ✅ LangSmith 프로젝트: 'LangChain-prantice'
    ✅ LangSmith API Key: 설정됨
    -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.
    ```

In [46]:
import os
from myrag import PDFRAG                        # local 임베딩 버전으로 수정한 myrag.py 불러오기
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

# API 키 확인
from dotenv import load_dotenv

GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY2")

if not os.getenv("GOOGLE_API_KEY2"):
    os.environ["GOOGLE_API_KEY2"] = input("Enter your GOOGLE_API_KEY2: ")

if "GOOGLE_API_KEY2" not in os.environ:
    print("❌ 경고: GOOGLE_API_KEY2 환경 변수가 설정되지 않았습니다. 반드시 설정해야 gemini LLM이 작동합니다.")

In [47]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm2 = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", temperature=0, api_key=os.getenv("GOOGLE_API_KEY2"))

print("✅ gemini-2.5-flash-lite 성공!")

✅ gemini-2.5-flash-lite 성공!


<small>

* **`✅ 성공!`** 

  * *`API 할당량 부족으로 두번쨰 계정으로 다시 시도`*

In [48]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm2 = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", temperature=0, api_key=os.getenv("GOOGLE_API_KEY2"))

rag = PDFRAG(
    "../15_Evaluations/data/SPRI_AI_Brief_2023년12월호_F.pdf",
    llm2,
)

print("✅ PDFRAG 생성 성공!")

✅ 문서 로드 완료: 23개 페이지
✅ 문서 분할 완료: 43개 청크
✅ 임베딩 모델 로드: all-MiniLM-L6-v2
✅ 벡터스토어 생성 완료
✅ PDFRAG 생성 성공!


<small>

* **`PDFRAG 생성`** - (`4.9s`)

    ```markdown

    ✅ 문서 로드 완료: 23개 페이지
    ✅ 문서 분할 완료: 43개 청크
    ✅ 임베딩 모델 로드: all-MiniLM-L6-v2
    ✅ 벡터스토어 생성 완료
    ✅ PDFRAG 생성 성공!

    ```

In [49]:
# 검색기(retriever) 생성
retriever = rag.create_retriever()

✅ 검색기 생성 완료 (k=4)


<small>

* **`✅ 검색기 생성 완료 (k=4)`**

In [50]:
# 체인(chain) 생성
chain = rag.create_chain(retriever)

✅ RAG 체인 생성 완료


<small>

* **`✅ RAG 체인 생성 완료`**

In [51]:
# 질문
answer = chain.invoke("삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?")
print(answer)

E0000 00:00:1760508400.597600 9355863 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


삼성전자가 자체 개발한 생성형 AI의 이름은 '삼성 가우스'입니다.


<small>

* **`삼성전자가 자체 개발한 생성형 AI의 이름은 '삼성 가우스'입니다.`** - (`2.0s`)

In [52]:
# 질문에 대한 답변하는 함수를 생성
def ask_question(inputs: dict):
    return {"answer": chain.invoke(inputs["question"])}

# 사용자 질문 예시
llm_answer = ask_question(
    {"question": "삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?"}
)

llm_answer

{'answer': "삼성전자가 자체 개발한 생성형 AI의 이름은 '삼성 가우스'입니다."}

<small>

* **`ask_question()`** - (`1.2s`)

    ```python

        {'answer': "삼성전자가 자체 개발한 생성형 AI의 이름은 '삼성 가우스'입니다."}

    ```

In [55]:
# evaluator prompt 출력을 위한 함수
def print_evaluator_prompt(evaluator):
    return evaluator.evaluator.prompt.pretty_print()

In [56]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator
from langchain_google_genai import ChatGoogleGenerativeAI

# qa 평가자 생성
qa_evalulator = LangChainStringEvaluator(
    "qa",
    config={
        "llm":ChatGoogleGenerativeAI(
            model="gemini-2.5-flash-lite", 
            temperature=0, 
            google_api_key=os.getenv("GOOGLE_API_KEY2"))
        }
)

# 프롬프트 출력
print_evaluator_prompt(qa_evalulator)

You are a teacher grading a quiz.
You are given a question, the student's answer, and the true answer, and are asked to score the student answer as either CORRECT or INCORRECT.

Example Format:
QUESTION: question here
STUDENT ANSWER: student's answer here
TRUE ANSWER: true answer here
GRADE: CORRECT or INCORRECT here

Grade the student answers based ONLY on their factual accuracy. Ignore differences in punctuation and phrasing between the student answer and true answer. It is OK if the student answer contains more information than the true answer, as long as it does not contain any conflicting statements. Begin! 

QUESTION: {query}
STUDENT ANSWER: {result}
TRUE ANSWER: {answer}
GRADE:


<small>

* **`qa_evaluator`** - (`0.1s`)

    ```markdown

    You are a teacher grading a quiz.
    You are given a question, the student's answer, and the true answer, and are asked to score the student answer as either CORRECT or INCORRECT.

    Example Format:
    QUESTION: question here
    STUDENT ANSWER: student's answer here
    TRUE ANSWER: true answer here
    GRADE: CORRECT or INCORRECT here

    Grade the student answers based ONLY on their factual accuracy. Ignore differences in punctuation and phrasing between the student answer and true answer. It is OK if the student answer contains more information than the true answer, as long as it does not contain any conflicting statements. Begin! 

    QUESTION: *{query}*
    STUDENT ANSWER: *{result}*
    TRUE ANSWER: *{answer}*
    GRADE:

    ```

* **`평가 진행` → 출력한 `URL` 이동 → `결과 확인하기`**

In [57]:
dataset_name = "RAG_EVAL_DATASET"

# 평가 실행
experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=[qa_evalulator],
    experiment_prefix="RAG_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "QA Evaluator 를 활용한 평가",
    },
)

View the evaluation results for experiment: 'RAG_EVAL-7ff2d7c9' at:
https://smith.langchain.com/o/2c3342d3-1170-4ffa-86fd-f621199e0b9c/datasets/420dd308-2ebd-44c9-8ce8-9aff3886dc8e/compare?selectedSessions=6d370b7d-2aa7-4b8e-9d38-78c005eea377




0it [00:00, ?it/s]E0000 00:00:1760508510.914989 9404755 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1760508510.921383 9404756 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
5it [00:04,  1.02it/s]


<small>

* 결과 확인하기

  * `RAG_EVAL` / *`COMPILED`* - (*`45s`*)
  * ![`RAG_EVAL_COMPILED`](../15_Evaluations/assets/RAG_EVAL_1.png)

  * 

  * `RAG_EVAL` / *`CORRECT=1`, `INCORRECT=4`* - (*`7.8s`*)
  * ![`RAG_EVAL_MORE_INCORRECT`](../15_Evaluations/assets/RAG_EVAL_2.png)

---

* **`RAG`** 시스템 개선하기

  * **a. `Retriever` 개선하기**
  * **b. `Chunk Size` 조정하기**

In [1]:
# ========================================
# Chunk Size 조정
# ========================================
from myrag2 import PDFRAG
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv

load_dotenv() 

# Chunk Size 조정
rag2 = PDFRAG(
    "../15_Evaluations/data/SPRI_AI_Brief_2023년12월호_F.pdf",
    ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-lite", 
        temperature=0, 
        google_api_key=os.getenv("GOOGLE_API_KEY2")),
)

✅ 문서 로드 완료: 23개 페이지
✅ 문서 분할 완료: 72개 청크
✅ 임베딩 모델 로드: all-MiniLM-L6-v2
✅ 벡터스토어 생성 완료


<small>

* **`rag2 생성`** - (`12.2s`)

    ```mardkwon

    ✅ 문서 로드 완료: 23개 페이지
    ✅ 문서 분할 완료: 72개 청크
    ✅ 임베딩 모델 로드: all-MiniLM-L6-v2
    ✅ 벡터스토어 생성 완료

    ```

In [3]:
# ========================================
# Top-K 증가
# ========================================

# 개선
retriever2 = rag2.create_retriever()        # myrag2.py 불러오기

# 체인 재생성
chain2 = rag2.create_chain(retriever2)

✅ 검색기 생성 완료 
✅ RAG 체인 생성 완료


<small>

* **`✅ 검색기 생성 완료`**

* **`✅ RAG 체인 생성 완료`**

In [4]:
# 질문에 대한 답변 생성
chain2.invoke("삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
E0000 00:00:1760512905.700200 9492174 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


'I am sorry, but the provided context does not contain the answer to your question.'

<small>

* **`'삼성전자가 자체 개발한 생성형 AI의 이름은 제공된 문서에서 찾을 수 없습니다.'`** - (`1.4s`)

In [65]:
# 질문에 대한 답변하는 함수를 생성
def ask_question(inputs: dict):
    return {"answer": chain2.invoke(inputs["question"])}

# 사용자 질문 예시
llm_answer = ask_question(
    {"question": "삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?"}
)

llm_answer

{'answer': '삼성전자가 자체 개발한 생성형 AI의 이름은 제공된 문서에서 찾을 수 없습니다.'}

<small>

* **`ask_question()`** - (`1.3s`)

    ```python

    {'answer': '삼성전자가 자체 개발한 생성형 AI의 이름은 제공된 문서에서 찾을 수 없습니다.'}

    ```

In [5]:
# evaluator prompt 출력을 위한 함수
def print_evaluator_prompt(evaluator):
    return evaluator.evaluator.prompt.pretty_print()

In [6]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator
from langchain_google_genai import ChatGoogleGenerativeAI


# qa 평가자 생성
qa_evalulator = LangChainStringEvaluator(
    "qa",
    config={
        "llm":ChatGoogleGenerativeAI(
            model="gemini-2.5-flash-lite", 
            temperature=0, 
            google_api_key=os.getenv("GOOGLE_API_KEY2"))
        }
)


# 프롬프트 출력
print_evaluator_prompt(qa_evalulator)

You are a teacher grading a quiz.
You are given a question, the student's answer, and the true answer, and are asked to score the student answer as either CORRECT or INCORRECT.

Example Format:
QUESTION: question here
STUDENT ANSWER: student's answer here
TRUE ANSWER: true answer here
GRADE: CORRECT or INCORRECT here

Grade the student answers based ONLY on their factual accuracy. Ignore differences in punctuation and phrasing between the student answer and true answer. It is OK if the student answer contains more information than the true answer, as long as it does not contain any conflicting statements. Begin! 

QUESTION: {query}
STUDENT ANSWER: {result}
TRUE ANSWER: {answer}
GRADE:


In [71]:
dataset_name = "RAG_EVAL_DATASET"

# 평가 실행
experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=[qa_evalulator],
    experiment_prefix="RAG_EVAL_2",
    # 실험 메타데이터 지정
    metadata={
        "variant": "myrag2.py 반영",
    },
)

View the evaluation results for experiment: 'RAG_EVAL_2-00c422bd' at:
https://smith.langchain.com/o/2c3342d3-1170-4ffa-86fd-f621199e0b9c/datasets/420dd308-2ebd-44c9-8ce8-9aff3886dc8e/compare?selectedSessions=cfb85525-ad18-4c76-9e47-4b2c1beb95fa




0it [00:00, ?it/s]E0000 00:00:1760510512.220081 9442399 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
5it [00:04,  1.12it/s]


<small>

* **개선 시도: `RAG_EVAL_2`**

  ```bash

  View the evaluation results for experiment: 'RAG_EVAL_2-00c422bd' at:
  https://smith.langchain.com/o/2c3342d3-1170-4ffa-86fd-f621199e0b9c/datasets/420dd308-2ebd-44c9-8ce8-9aff3886dc8e/compare?selectedSessions=cfb85525-ad18-4c76-9e47-4b2c1beb95fa


  0it [00:00, ?it/s]

  5it [00:04,  1.12it/s]

  ```

* 결과 확인하기

<br>

* 
  * `RAG_EVAL` / *`CORRECT=2`, `INCORRECT=3`* - (*`7.3s`*)
  * ![`RAG_EVAL_MORE_INCORRECT`](../15_Evaluations/assets/RAG_EVAL_3.png)

---

* **`RAG`** 시스템 개선하기_2

  * **c. `chunk_size` 조정**
  * **d. 잦은 커널 충돌 → `python` 파일로 실행하기**

* c.

  * `chunck_size` 조절 test
    * **`chunk_size` 지정 ❌** or **`chunk_size` = `4` → 검색 결과 없음**
      * *`'죄송합니다. 제공된 문서에서 삼성전자가 자체 개발한 생성형 AI의 이름에 대한 정보를 찾을 수 없습니다.'`*

    * **`chunk_size` = `7` → 검색 결과 ⭕️**
      * *`"삼성전자가 자체 개발한 생성형 AI의 이름은 '삼성 가우스'입니다."`*

  * [**`myrag4.py`**](../15_Evaluations/myrag4.py)로 정리

* d.
  * **`잦은 커널 충돌로 인한 문제` → `python` 파일로 실행 → 터미널에서 직접 확인하기**

  * [**`eval_script.py`**](../15_Evaluations/eval_script.py) 실행 → 터미널에서 결과 바로 확인

  * ![`RAG_EVAL_4`](../15_Evaluations/assets/RAG_EVAL_4.png)

<small>

```bash

      (가상환경)/15_Evaluations/eval_script.py

      🚀 RAG 시스템 초기화 시작...

      📂 스크립트 위치: ... /15_Evaluations

      📄 PDF 경로: ... /data/SPRI_AI_Brief_2023년12월호_F.pdf

      ✅ 파일 확인 완료!

      ✅ LLM 생성 완료: gemini-2.5-flash-lite

      ✅ PDF 로드 완료: 23 페이지

      ✅ 청크 분할 완료: 119 청크 (크기=300, 오버랩=50)


      ✅ LLM 생성 완료: gemini-2.5-flash-lite

      ✅ 청크 분할 완료: 119 청크 (크기=300, 오버랩=50)
      pytorch_model.bin: 100%|████████████████████████████████████████████████████████████████████| 2.27G/2.27G [00:03<00:00, 695MB/s]
      tokenizer_config.json: 100%|███████████████████████████████████████████████████████████████████| 444/444 [00:00<00:00, 1.92MB/s]
      sentencepiece.bpe.model: 100%|██████████████████████████████████████████████████████████████| 5.07M/5.07M [00:06<00:00, 781kB/s]
      tokenizer.json: 100%|██████████████████████████████████████████████████████████████████████| 17.1M/17.1M [00:06<00:00, 2.70MB/s]
      special_tokens_map.json: 100%|█████████████████████████████████████████████████████████████████| 964/964 [00:00<00:00, 3.61MB/s]
      config.json: 100%|██████████████████████████████████████████████████████████████████████████████| 191/191 [00:00<00:00, 628kB/s]

      ✅ 임베딩 모델 로드 완료: BAAI/bge-m3                                                                 | 0.00/191 [00:00<?, ?B/s]
      model.safetensors:   0%|                                                                  | 2.19M/2.27G [00:24<1:26:03, 439kB/s]✅ 벡터스토어 생성 완료: FAISS

      ✅ 검색기 생성 완료 (k=7, search_type=similarity)

      ✅ RAG 체인 생성 완료

      ==================================================
      🧪 테스트 실행...
      ==================================================

      View the evaluation results for experiment: 'RAG_EVAL_K7-3a35bd45' at:
      https://smith.langchain.com/o/2c3342d3-1170-4ffa-86fd-f621199e0b9c/datasets/420dd308-2ebd-44c9-8ce8-9aff3886dc8e/compare?selectedSessions=cdb9ea10-e3c1-47bc-bfe1-48f651364c28


      5it [00:14,  2.90s/it]
      5it [00:14,  2.18s/it]

      ==================================================
      ✅ 평가 완료!
      ==================================================

      결과: <ExperimentResults RAG_EVAL_K7-3a35bd45>

      model.safetensors: 100%|███████████████████████████████████████████████████████████████████| 2.27G/2.27G [02:14<00:00, 16.9MB/s]

```

---

* **`현재 상황 분석`**

  | 항목 | 개수 | 정확도 |
  |------|------|--------|
  | **CORRECT** | 3/5 | 60% |
  | **INCORRECT** | 2/5 | 40% |

* 진전
  * 이전: 1/5 (20%)
  * 현재: 3/5 (60%)
  * **향상: +200%!**

* **`RAG` 평가**

  * 어려움

    | 이유 | 설명 |
    |------|------|
    | **초기 정확도** | 20-40%는 정상 |
    | **반복 개선** | 60% → 80% → 90%로 점진적 향상 |
    | **완벽은 없음** | 100%는 거의 불가능 |

<br>

* 

  * 현재 단계

    ```markdown
        일반적인 RAG 평가 발전:

        1단계: 20-40% (초기)
        2단계: 50-60% (개선 후)  ←  여기!
        3단계: 70-80% (최적화)
        4단계: 80-90% (완성)
    ```

<br>

* **`향상 방법`**

  * **a. `k값 증가`**

  * **b. `chunk_size` 조정하기**

  * **c. `프롬프트 개선하기`**

  * **d. `다른 임베딩 모델 시도하기`**

---

#### **3) `Context 에 기반한 답변 Evaluator`**

* **`LangChainStringEvaluator("context_qa")`**: `LLM` 체인에 정확성을 판단하는 데 참조 **`"context"`** 를 사용 → 지시

* **`LangChainStringEvaluator("cot_qa")`**: 
  * `"cot_qa"` = `"context_qa"` 평가자와 유사
  * `but` 최종 판결을 결정하기 전에 `LLM` 의 `'추론'`을 사용하도록 지시한다는 점에서 차이 있음

In [11]:
# query_1 
# 질문에 대한 답변 생성
chain.invoke("삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?")

'참고 자료에 해당 정보가 없습니다.'

<small>

* **`chain.invoke()`** - (`0.6s`)

    ```markdown

    '주어진 문서에서 삼성전자가 자체 개발한 생성형 AI의 이름에 대한 정보는 찾을 수 없습니다.'

    ```

---

* **`디버깅하기`**

  * `RAG 체인`의 핵심 구성 요소 = **`문서 로드`, `분할`, `임베딩`, `검색`(Retrieval) 단계**

In [12]:
# 임베딩 및 리트리버 테스트

# 1. PDFRAG 객체 생성 시점까지 실행
# file_path = "../15_Evaluations/data/SPRI_AI_Brief_2023년12월호_F.pdf"
# llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", temperature=0)

rag = PDFRAG(
    file_path="../15_Evaluations/data/SPRI_AI_Brief_2023년12월호_F.pdf",
    llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", temperature=0))

# 2. 리트리버 객체 가져오기
retriever = rag.create_retriever()

# 3. 검색 테스트: 
#   -> 이 부분에서 오류가 발생하면 **임베딩/VectorStore** 문제
docs = retriever.invoke("삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?")

# 4. 결과 출력: Document 객체 리스트가 출력되어야 함
print(f"검색된 문서 개수: {len(docs)}")
print(docs[0].page_content)

E0000 00:00:1760439003.197106 7875803 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


>>> BAAI/bge-small-en-v1.5 모델을 사용하여 임베딩을 생성 중...
>>> 벡터 데이터베이스 생성이 완료되었습니다.
검색된 문서 개수: 3
영국을 AI 안전 연구의 글로벌 허브로 확립하는 것을 목표로 함
∙영국 정부는 향후 10년간 연구소에 공공자금을 투자해 연구를 지원할 계획으로, 연구소는 △첨단 AI 시스템 
평가 개발과 시행 △AI 안전 연구 촉진 △정보 교류 활성화를 핵심 기능으로 함
n (첨단 AI 시스템 평가 개발과 시행) 시스템의 안전 관련 속성을 중심으로 안전과 보안 기능을 이해
하고 사회적 영향을 평가
∙평가 우선순위는 △사이버범죄 조장, 허위 정보 유포 등 악의적으로 활용될 수 있는 기능 △사회에 미치는


<small>

* **`RAG 초기화 완료: 로컬 임베딩과 LangChain LLM 사용`** - (`11.3s`)

    ```markdown

    검색된 문서 개수: 4
    1. 정책/법제  
    2. 기업/산업 
    3. 기술/연구 
    4. 인력/교육
    영국 과학혁신기술부, AI 안전 연구소 설립 발표
    n 영국 과학혁신기술부가 첨단 AI 시스템에 대한 평가를 통해 안전성을 보장하기 위한 AI 
    안전 연구소를 설립한다고 발표
    n AI 안전 연구소는 핵심 기능으로 첨단 AI 시스템 평가 개발과 시행, AI 안전 연구 촉진, 
    정보교류 활성화를 추진할 계획
    KEY Contents

    ```

<small>

* 오류 발생 원인 추론 - *(주로 `LLM` 호출 단계에 많음)*

  * `LLM API Key` or 환경 변수 문제
    * `llm 객체 초기화` 단계에서 `API key`가 제대로 설정되지 않았을 경우
    * 디버깅에서 `llm 객체` 생성 과정에서 해결되었을 가능성
    
    ```python

    # 환경변수 설정 확인
    # ChatGoogleGenerativeAI()의 경우
    os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEY"

    # ChatOpenAI()의 경우
    os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

    # llm 객체 초기화
    llm = ChatGoogleGenerativeAI(...)
    # 혹은
    llm = ChatOpenAI(...)

    ```

<br>

* 
  * `Pydantic` 버전 충돌 문제
    * `myrag.py` 파일 = `Pydantic v1` 환경 작성된 코드 → 현재 실행 환경이 `Pydantic v2` 요구한다면 실패
    * 로컬 임베딩 모델인 `HuggingFaceEmbeddings`로 교체 → 필요한 패키지 설치되었는지 확인 

    ```bash

    # 패키지 설치 필요한 경우
    pip install sentence-transformers   

    # 패키지 확인 필요한 경우
    pip show sentence-transformers

    ```

  * * 설치 확인 과정에서 해결되었을 가능성

<br>

* 
  * `템플릿 변수 누락` or `포맷 오류`
    * `retriever`가 검색한 `context`는 잘 나왔지만, **`question` 키에 해당하는 질문이 `chain`에 전달되지 않아** 프롬프트 템플릿 포맷팅 실패 가능성

    ```python
    # 정상
    rag_chain.invoke(question_string)

    # 실패한 포맷
    rag_chain.invoke()

    # 디버깅
    # 정상적인 문자열로 포맷팅
    docs = retriever.invoke("삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?")
    
    ```

    * 디버깅 코드 실행 **→ `명확하게 입력 문자열 전달`** → 입력 누락 해결되었을 가능성

---

In [13]:
from myrag import PDFRAG
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

# OpenAI LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

/Users/jay/.pyenv/versions/lc_env/lib/python3.13/site-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'validate_by_name'
  warnings.warn(message, UserWarning)


PydanticUserError: The `__modify_schema__` method is not supported in Pydantic v2. Use `__get_pydantic_json_schema__` instead in class `SecretStr`.

For further information visit https://errors.pydantic.dev/2.11/u/custom-json-schema

---

* **`ask_question()` 함수 생성하기** 

  * 입력 = `inputs` = `딕셔너리`
  * 출력 = `answer` = `딕셔너리`를 반환

In [35]:
# 질문에 대한 답변하는 함수를 생성
def ask_question(inputs: dict):
    return {"answer": chain.invoke(inputs["question"])}

In [36]:
# 사용자 질문 예시
llm_answer = ask_question(
    {"question": "삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?"}
)
llm_answer

{'answer': '주어진 문서에서 삼성전자가 자체 개발한 생성형 AI의 이름에 대한 정보는 찾을 수 없습니다.'}

In [ ]:
import pandas as pd

# 질문과 답변 목록
inputs = [
    "삼성전자가 만든 생성형 AI의 이름은 무엇인가요?",
    "미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행정명령을 발표한 날은 언제인가요?",
    "코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.",
]

# 질문에 대한 답변 목록
outputs = [
    "삼성전자가 만든 생성형 AI의 이름은 삼성 가우스 입니다.",
    "2023년 10월 30일 미국 바이든 대통령이 행정명령을 발표했습니다.",
    "코히어의 데이터 출처 탐색기는 AI 모델 훈련에 사용되는 데이터셋의 출처와 라이선스 상태를 추적하고 투명성을 확보하기 위한 플랫폼입니다. 12개 기관과 협력하여 2,000여 개 데이터셋의 출처 정보를 제공하며, 개발자들이 데이터의 구성과 계보를 쉽게 파악할 수 있게 돕습니다.",
]

In [ ]:
# 질문과 답변 쌍 생성
qa_pairs = [{"question": q, "answer": a} for q, a in zip(inputs, outputs)]

In [ ]:
# 데이터프레임으로 변환
df = pd.DataFrame(qa_pairs)

In [ ]:
# 데이터프레임 출력
df.head()

<small>

* `df.head()`

  * ![df.head()](../15_Evaluations/assets/df_head_2.png)

##### **`➁ 이전에서 생성한`** ***`Synthetic Dataset`*** **`활용하기`**

* 업로드했던 `HuggingFace Dataset` 활용하기

* 사전에 `VS Code` 터미널에 설치할 것

```bash
            pip install -qU datasets
```

In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset
import os

# huggingface Dataset에서 repo_id로 데이터셋 다운로드
dataset = load_dataset(
    "livemylife23/rag-synthetic-dataset-korean",   # 데이터셋 이름
    token=os.environ["HUGGINGFACEHUB_API_TOKEN"],  # private 데이터인 경우 필요
)

<small>

* 다운로드 완료 - (`7.1s`)

  * ![hugginface_dataset_download](../15_Evaluations/assets/hugging_face_download.png)

In [ ]:
# 데이터셋에서 split 기준으로 조회
huggingface_df = dataset["train"].to_pandas()
huggingface_df.head()

<small>

  * `huggingface_df.head()`

    * ![huggingface_df.head()](../15_Evaluations/assets/df_head_3.png)

---

#### **2) `LangSmith 테스트를 위한 Dataset 생성`**

* **`Dataset & Testing`에 새로운 데이터셋 생성하기**

<small>

  * ![langsmith에 새로운 데이터셋 생성하기](../15_Evaluations/assets/eval-06.png)

* `csv` 파일에서 `LangSmith UI` 사용 → 직접 데이터셋 생성 가능

* *참고: [`LangSmith UI 문서`](https://docs.langchain.com/langsmith/manage-datasets)*

In [ ]:
from langsmith import Client

client = Client()
dataset_name = "RAG_EVAL_DATASET"

In [ ]:
# 데이터셋 생성 함수

def create_dataset(client, dataset_name, description=None):
    for dataset in client.list_datasets():
        if dataset.name == dataset_name:
            return dataset

    dataset = client.create_dataset(
        dataset_name=dataset_name,
        description=description,
    )
    return dataset

In [ ]:
# 데이터셋 생성
dataset = create_dataset(client, dataset_name)          # 2.9s

In [ ]:
# 생성된 데이터셋에 예제 추가

client.create_examples(
    inputs=[{"question": q} for q in df["question"].tolist()],
    outputs=[{"answer": a} for a in df["answer"].tolist()],
    dataset_id=dataset.id,
)

<small>

* **`client.creat_examples() 추가하기`** - (`0.1s`)

    ```python
    {'example_ids': ['8d34ad4e-e3be-47ca-9a6a-9929608b60ca',
    '71841ffc-cfbe-4e05-b0b0-ef85e8a6ebd4',
    'e0d645a9-6d97-41da-8d45-cc5e8d6f0a37'],
    'count': 3}
    ```

* **`데이터셋에 예제 나중에 추가 가능`**

In [ ]:
# 새로운 질문 목록
new_questions = [
    "삼성전자가 만든 생성형 AI의 이름은 무엇인가요?",
    "구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?",
]

In [ ]:
# 새로운 답변 목록
new_answers = [
    "삼성전자가 만든 생성형 AI의 이름은 테디노트 입니다.",
    "사실이 아닙니다. 구글은 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억 달러를 우선 투자하고 향후 15억 달러를 추가로 투자하기로 했습니다.",
]

In [ ]:
# UI에서 업데이트된 버전 확인

client.create_examples(
    inputs=[{"question": q} for q in new_questions],
    outputs=[{"answer": a} for a in new_answers],
    dataset_id=dataset.id,
)

<small>

* **`UI에서 업데이트된 버전 확인하기`** - (`0.1s`)

    ```python
    {'example_ids': ['7080404f-878d-4e22-9256-70a5d1b86ab3',
    'b8599d26-9990-4153-8a92-0dc779087f2c'],
    'count': 2}
    ```

* **✓ `데이터셋 준비 완료됨`**

---

* next: ***`05. LLM-as-Judge`***

---